In [2]:
import os
import sys

# os.path.join(os.path.dirname(__file__)
sys.path.append("../")  # location of config file
sys.path.append("../../") # working directory inside exploration
sys.path.append("../../../") 



import config

os.chdir("../../")

import glob
import numpy as np
from smartprint import smartprint as sprint
from preprocessing.ProcessRaw import ProcessRaw
from smartprint import smartprint as sprint
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# ! rm -rf dist_plots
# ! mkdir dist_plots

In [ ]:
#########################################################################
#################  tryout: smoothing and thresholds  ####################
#########################################################################
# ! pip install peakdetect==1.2

import matplotlib.patches as patches

def compute_temp_dist(fileserial_num_list, L, plt):
    random.shuffle(fileserial_num_list)
        
    distances = []
    
    data_cache = {}
    sequential_delete = []
    # sprint (int(np.random.rand() * N), len(fileserial_num_list))

    # choose query point somewhere near the centre of the dataset
    # so that we can compute distances on both sides (temporally)
    query_index = np.inf
    while (not 6000 < query_index < 8000):
        n = int(np.random.rand() * len(fileserial_num_list))
        query_point = np.load(fileserial_num_list[n])
        query_index = int(fileserial_num_list[n].split("/")[-1].split("_x.npy")[0].split("-")[-1])
        # sprint (query_index)
        
    
    

    temporal_distances = []
    # query_index = int(fileserial_num_list[n].split("/")[-1].split("_x.npy")[0].split("-")[-1])

    for i in tqdm(range(1, len(fileserial_num_list)), desc = "computing distance to\
                                        random neighbours"):
        neighbour_x_array = np.load(fileserial_num_list[i])
        
        if "_x.npy" not in fileserial_num_list[i]:
            raise Exception ("Wrong file supplied; we should not have _y files\n since we are looking for n-hood of x")
            
        neighbour_index = int(fileserial_num_list[i].split("/")[-1].split("_x.npy")[0].split("-")[-1])
        
        distances.append(np.max(np.abs(query_point - neighbour_x_array))) 
        temporal_distances.append((query_index - neighbour_index))

        
    # return temporal_distances, distances
    temporal_distances, distances, indices = zip(*sorted(zip(temporal_distances, distances, list(range(1, len(fileserial_num_list))))))
    # sprint (temporal_distances[:5])
    # sprint ([fileserial_num_list[i] for i in indices][:5])
    # sprint (distances[:16])
    
    nn = temporal_distances.index(0)

    # plt.plot(temporal_distances[:96*7], np.convolve(distances[:96*7], [1/1]*1,"same"))
    # plt.plot(temporal_distances[:64], np.convolve(distances[:64], [1/8]*8,"same"))
    plt.plot(temporal_distances[nn - L : nn + L], distances[nn - L: nn + L], alpha=0.1)
    
    return distances[nn - L: nn + L], plt

from peakdetect import peakdetect

smooth_window = np.nan # no longer being used
for L in [1000]:
    for cityname in ["london"]:#, "Madrid", "Melbourne"]:
        for scale in config.scales_def:
            for i_o_length in [1]:
                for pred_horiz in [1] :# list(range(1, 8)): # [1]:
                    plotted_dist_list = []
                    plt.clf()

                    for run_n_times in range(10):
                        ProcessRaw(cityname=cityname, i_o_length=i_o_length, prediction_horizon=pred_horiz, grid_size=scale)
                        prefix = ProcessRaw.file_prefix(cityname, i_o_length, pred_horiz, scale)

                        file_list = glob.glob("../train_data_all_cities/" + prefix + "/*_x.npy")
                        distances, plt = compute_temp_dist(file_list, L, plt)
                        plotted_dist_list.append(distances)
                    
                    # plt.show()
                    plt.clf()
                    plt.title("Distance vs. temporal distance\n"+prefix)
                    
                    median_plot = np.median(np.array(plotted_dist_list), axis=0)
                    plt.plot(np.arange(-L, L, 1), median_plot, label="Median plot", alpha=0.5)
                    # plt.plot(np.arange(-L, L, 1)[smooth_window//2:-smooth_window//2], \
                    #          np.convolve(np.median(np.array(plotted_dist_list), axis=0), \
                    #         [1/smooth_window]*smooth_window, "same")[smooth_window//2:-smooth_window//2]\
                    #          , label="Median plot-smooth", alpha=1)
                    
                    
                    
                    plt.ylim(0, 6000)
                    plt.xlabel("1 unit: 15 minutes")
                    plt.legend()
                    plt.savefig("dist_plots/"+prefix+"-tempdist-"+str(L)+".png")
                    plt.clf()       
                    

                    # compute peaks
                    peaks = peakdetect(median_plot, lookahead=50) 
                    higherPeaks = np.array(peaks[0])
                    lowerPeaks = np.array(peaks[1])
                    plt.plot(np.arange(-L, L, 1), median_plot)
                    plt.plot(higherPeaks[:,0]-L, higherPeaks[:,1], 'ro')
                    plt.plot(lowerPeaks[:,0]-L, lowerPeaks[:,1], 'ko')
                    plt.xticks(np.arange(-10, 10) * 96, ["day_"+str(x) for x in range(-10,10)],rotation=90)
                    plt.title("Distance vs. temporal distance\n"+prefix+"-zoom1")
                    plt.savefig("dist_plots/"+"Distance vs. temporal distance\n"+prefix+"-zoom1"+".png")
                    plt.clf()


                    peaks = peakdetect(median_plot, lookahead=50) 
                    higherPeaks = np.array(peaks[0])
                    lowerPeaks = np.array(peaks[1])
                    plt.plot(np.arange(-L, L, 1), median_plot)
                    plt.plot(higherPeaks[:,0]-L, higherPeaks[:,1], 'ro')
                    plt.plot(lowerPeaks[:,0]-L, lowerPeaks[:,1], 'ko')
                    plt.xticks(np.arange(-10, 10) * 96, ["day_"+str(x) for x in range(-10,10)],rotation=90)
                    plt.xlim(-200, 200)
                    plt.title("Distance vs. temporal distance\n"+prefix +"-zoom2")
                    plt.savefig("dist_plots/"+"Distance vs. temporal distance\n"+prefix +"-zoom2" + ".png")                    
                    plt.clf()


                    peaks = peakdetect(median_plot, lookahead=50) 
                    higherPeaks = np.array(peaks[0])
                    lowerPeaks = np.array(peaks[1])
                    plt.plot(np.arange(-L, L, 1), median_plot)
                    plt.plot(higherPeaks[:,0]-L, higherPeaks[:,1], 'ro')
                    plt.plot(lowerPeaks[:,0]-L, lowerPeaks[:,1], 'ko')
                    plt.xticks(np.arange(-20, 20) * 4, ["hour_"+str(x) for x in range(-20,20)],rotation=90)
                    plt.xlim(-96, 96)
                    plt.title("Distance vs. temporal distance\n"+prefix+ "-zoom3")
                    plt.savefig("dist_plots/"+"Distance vs. temporal distance\n"+prefix+ "-zoom3"+".png")                    
                    plt.clf()


                    where_min = np.argmin(lowerPeaks[:,1])
                    x1 = lowerPeaks[where_min-1, 0]
                    x2 = lowerPeaks[where_min+1, 0]
                    y1 = lowerPeaks[where_min-1, 1]
                    y2 = lowerPeaks[where_min+1, 1]

                    # reduce a small value to avoid the immediate next step (noise)
                    thresh = min(y1, y2) - 5

                    plt.plot(np.arange(-L, L, 1), [thresh] * (2 * L), label="threshold")

                    peaks = peakdetect(median_plot, lookahead=50) 
                    higherPeaks = np.array(peaks[0])
                    lowerPeaks = np.array(peaks[1])

                    plt.scatter([x1-L, x2-L], [y1, y2], color='b', marker='*', alpha=0.5, s=300)
                    plt.plot(np.arange(-L, L, 1), median_plot, label="median_plot")
                    plt.plot(higherPeaks[:,0]-L, higherPeaks[:,1], 'ro')
                    plt.plot(lowerPeaks[:,0]-L, lowerPeaks[:,1], 'ko')
                    plt.xticks(np.arange(-20, 20) * 4, ["hour_"+str(x) for x in range(-20,20)],rotation=90)
                    plt.xlim(-96, 96)


                    x_vals_for_below_thresh = np.argwhere(np.diff(np.sign(median_plot - thresh))).flatten()
                    left = x_vals_for_below_thresh.min() - L
                    bottom = 0
                    width = x_vals_for_below_thresh.max() - x_vals_for_below_thresh.min()
                    height = thresh

                    rect = patches.Rectangle((left, bottom), width, height, linewidth=1, edgecolor='r', facecolor='gray', alpha=0.3)
                    # rect = patches.Rectangle((50, 100), 40, 30, linewidth=1, edgecolor='r', facecolor='none')
                    ax = plt.gca()
                    # Add the patch to the Axes
                    ax.add_patch(rect)
                    plt.legend()
                    plt.title("Distance vs. temporal distance\n"+prefix+ "-zoom3" + " \n#datapoints="+str(width), fontsize=8)
                    plt.savefig("dist_plots/"+"Distance vs. temporal distance\n"+prefix+ "-zoom3" + " \n#datapoints="+str(width)+".png")
                    plt.clf()

london_45_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 1996.17it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-45-:   0%|          | 0/215 [00:00<?, ?it/s]
computing distance to                                        random neighbours: 100%|██████████| 16559/16559 [00:03<00:00, 4901.59it/s]
london_45_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 14629.71it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-45-:   0%|          | 0/215 [00:00<?, ?it/s]
computing distance to                                        random neighbours: 100%|██████████| 16559/16559 [00:03<00:00, 4900.74it/s]
london_45_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 8276.97it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-45-:   0%|          | 0/215 [00:00<?, ?it/s]

['../train_data_all_cities/london-1-1-45-/london-1-1-45-13346_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-12426_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-13023_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-15303_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-7305_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-2012_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-4537_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-3832_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-1589_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-14047_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-506_x.npy']

['../train_data_all_cities/london-1-1-45-/london-1-1-45-787_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-134_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-14457_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-15048_x.npy',
 '../train_data_all_cities/london-1-1-45-/london-1-1-45-9595_x.npy']

In [8]:
!pwd

/home/niskumar/NeurIPS2022-traffic4cast/exploration
